# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [192]:
%load_ext autoreload
%autoreload 2

In [54]:
import helper


# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [55]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [56]:
import collections


english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [57]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    tokenized_sentences = tokenizer.texts_to_sequences(x)

    return tokenized_sentences, tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'lexicography': 18, 'study': 21, 'prize': 5, 'a': 1, 'my': 7, 'lazy': 8, 'this': 16, 'short': 9, 'jumps': 10, 'over': 17, 'by': 11, 'fox': 12, 'sentence': 13, 'of': 4, 'won': 15, 'is': 20, 'the': 2, 'dog': 14, 'brown': 19, 'quick': 3, 'jove': 6}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [2, 3, 19, 12, 10, 17, 2, 8, 14]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [11, 6, 7, 3, 21, 4, 18, 15, 1, 5]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [16, 20, 1, 9, 13]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [58]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    return pad_sequences(x,padding='post',maxlen=length)

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 2  3 19 12 10 17  2  8 14]
  Output: [ 2  3 19 12 10 17  2  8 14  0]
Sequence 2 in x
  Input:  [11  6  7  3 21  4 18 15  1  5]
  Output: [11  6  7  3 21  4 18 15  1  5]
Sequence 3 in x
  Input:  [16 20  1  9 13]
  Output: [16 20  1  9 13  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [59]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

Data Preprocessed


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [60]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [61]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

In [83]:
from keras.layers import GRU, Input, Dense, TimeDistributed, LSTM
from keras.models import Model, Sequential
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size, learning_rate=0.1):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    model = Sequential()
    model.add(LSTM(output_sequence_length, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(french_vocab_size)))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_simple_model(simple_model)

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1, # Add a code for padding
    len(french_tokenizer.word_index) + 1,
    learning_rate=0.05 # Lowered learning rate for better fit
)

print(simple_rnn_model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 21, 21)            1932      
_________________________________________________________________
time_distributed_14 (TimeDis (None, 21, 346)           7612      
Total params: 9,544.0
Trainable params: 9,544
Non-trainable params: 0.0
_________________________________________________________________
None


In [84]:
simple_rnn_model.fit(
    tmp_x, 
    preproc_french_sentences, 
    batch_size=1024, epochs=10, validation_split=0.2
)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 8s - loss: 5.3029 - acc: 0.4281 - val_loss: 5.7866 - val_acc: 0.4688
Epoch 2/10
110288/110288 [==============================] - 7s - loss: 5.4413 - acc: 0.4681 - val_loss: 5.4483 - val_acc: 0.4617
Epoch 3/10
110288/110288 [==============================] - 7s - loss: 5.3247 - acc: 0.4607 - val_loss: 5.1004 - val_acc: 0.4375
Epoch 4/10
110288/110288 [==============================] - 7s - loss: 5.2188 - acc: 0.4305 - val_loss: 5.3482 - val_acc: 0.4160
Epoch 5/10
110288/110288 [==============================] - 7s - loss: 5.5239 - acc: 0.4012 - val_loss: 5.4122 - val_acc: 0.4390
Epoch 6/10
110288/110288 [==============================] - 7s - loss: 5.2289 - acc: 0.4099 - val_loss: 5.2563 - val_acc: 0.3387
Epoch 7/10
110288/110288 [==============================] - 8s - loss: 5.2567 - acc: 0.3865 - val_loss: 5.1239 - val_acc: 0.3681
Epoch 8/10
110288/110288 [====================

In [131]:
# Print prediction(s)

def preview_sentence(sentence_index,input_x,model):
    print('ENGLISH: ',english_sentences[sentence_index])
    print('FRENCH: ',
        logits_to_text(
            model.predict(input_x[sentence_index:sentence_index+1])[0],
            french_tokenizer
        )
    )
    pass

preview_sentence(13,tmp_x,simple_rnn_model)


ENGLISH:  that cat was my most loved animal .
FRENCH:  leur leur leur leur leur leur leur redoutã© aimã© lion aoã»t <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [182]:
from keras.layers.embeddings import Embedding


def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    model = Sequential()
    model.add(Embedding(
        input_length = input_shape[1],
        input_dim=english_vocab_size,
        output_dim = 10 # Random value for dense vector
    ))
    
    model.add(LSTM(
        output_sequence_length,
        input_shape = input_shape[1:],
        return_sequences = True           
    ))
    
    model.add(TimeDistributed(Dense(french_vocab_size)))

    model.summary()

    model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

    return model

tests.test_embed_model(embed_model)

# TODO: Reshape the input
max_length = max(preproc_english_sentences.shape[1],preproc_french_sentences.shape[1])
padded_x = pad(preproc_english_sentences, max_length)

embed_rnn = embed_model(
    padded_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) + 1
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_35 (Embedding)     (None, 21, 10)            1990      
_________________________________________________________________
lstm_62 (LSTM)               (None, 21, 21)            2688      
_________________________________________________________________
time_distributed_54 (TimeDis (None, 21, 344)           7568      
Total params: 12,246.0
Trainable params: 12,246
Non-trainable params: 0.0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_36 (Embedding)     (None, 21, 10)            2000      
_________________________________________________________________
lstm_63 (LSTM)               (None, 21, 21)            2688      
_________________________________________________________________
ti

In [135]:
# TODO: Train the neural network
embed_rnn.fit(padded_x,preproc_french_sentences, batch_size=4096, epochs=10, validation_split=0.2)


Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 3s - loss: 1.7909 - acc: 0.5956 - val_loss: 1.6991 - val_acc: 0.6081
Epoch 2/10
110288/110288 [==============================] - 4s - loss: 1.7526 - acc: 0.6020 - val_loss: 1.8104 - val_acc: 0.5988
Epoch 3/10
110288/110288 [==============================] - 4s - loss: 1.8042 - acc: 0.5898 - val_loss: 1.8109 - val_acc: 0.5806
Epoch 4/10
110288/110288 [==============================] - 4s - loss: 1.7400 - acc: 0.6032 - val_loss: 1.7048 - val_acc: 0.6166
Epoch 5/10
110288/110288 [==============================] - 4s - loss: 1.7494 - acc: 0.6035 - val_loss: 1.6923 - val_acc: 0.6106
Epoch 6/10
110288/110288 [==============================] - 4s - loss: 1.6901 - acc: 0.6114 - val_loss: 1.6561 - val_acc: 0.6160
Epoch 7/10
110288/110288 [==============================] - 4s - loss: 1.6526 - acc: 0.6188 - val_loss: 1.6292 - val_acc: 0.6209
Epoch 8/10
110288/110288 [====================

In [134]:
# TODO: Print prediction(s)
preview_sentence(5,padded_x,embed_rnn)

ENGLISH:  his favorite fruit is the orange , but my favorite is the grape .
FRENCH:  son fruit est est la est mais aimã© prã©fã©rã© est la le <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [176]:
from keras.layers import Bidirectional


def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    model = Sequential()
    model.add(
        Bidirectional(
            LSTM(
                output_sequence_length,
                return_sequences=True
            ),
            input_shape=input_shape[1:],
            merge_mode='sum'
        )
    )

    model.add(TimeDistributed(Dense(french_vocab_size)))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(),
                  metrics=['accuracy'])

    model.summary()

    return model

tests.test_bd_model(bd_model)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_20 (Bidirectio (None, 21, 21)            3864      
_________________________________________________________________
time_distributed_48 (TimeDis (None, 21, 344)           7568      
Total params: 11,432.0
Trainable params: 11,432
Non-trainable params: 0.0
_________________________________________________________________


In [162]:

bd_rnn = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) + 1
)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_16 (Bidirectio (None, 21, 21)            3864      
_________________________________________________________________
time_distributed_46 (TimeDis (None, 21, 346)           7612      
Total params: 11,476.0
Trainable params: 11,476
Non-trainable params: 0.0
_________________________________________________________________


In [171]:
# TODO: Train and Print prediction(s)
bd_rnn.fit(tmp_x, preproc_french_sentences, batch_size=4096, epochs=10, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 15s - loss: 3.5667 - acc: 0.4445 - val_loss: 3.5406 - val_acc: 0.4444
Epoch 2/10
110288/110288 [==============================] - 14s - loss: 3.5480 - acc: 0.4457 - val_loss: 3.5411 - val_acc: 0.4443
Epoch 3/10
110288/110288 [==============================] - 14s - loss: 3.5450 - acc: 0.4462 - val_loss: 3.5324 - val_acc: 0.4521
Epoch 4/10
110288/110288 [==============================] - 14s - loss: 3.5505 - acc: 0.4491 - val_loss: 3.5241 - val_acc: 0.4438
Epoch 5/10
110288/110288 [==============================] - 14s - loss: 3.5730 - acc: 0.4426 - val_loss: 3.5871 - val_acc: 0.4478
Epoch 6/10
110288/110288 [==============================] - 14s - loss: 3.5691 - acc: 0.4465 - val_loss: 3.5309 - val_acc: 0.4535
Epoch 7/10
110288/110288 [==============================] - 14s - loss: 3.5394 - acc: 0.4434 - val_loss: 3.5173 - val_acc: 0.4487
Epoch 8/10
110288/110288 [=============

In [172]:
preview_sentence(3,tmp_x,bd_rnn)

ENGLISH:  the united states is sometimes mild during june , and it is cold in september .
FRENCH:  gã©nã©ralement lapin lapin est parfois lapin trop cours trop neigeux gã¨le favori veut favori <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [ ]:
from keras.layers import RepeatVector


def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    return None
tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [235]:
import project_tests as tests

def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    model = Sequential()

    model.add(Embedding(
        input_length = input_shape[1],
        input_dim=english_vocab_size,
        output_dim = 10 # Random value for dense vector
    ))

    model.add(
        Bidirectional(
            GRU(
                output_sequence_length,
                return_sequences=True
            ),
            merge_mode='sum'
        )
    )

    model.add(TimeDistributed(Dense(french_vocab_size)))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(),
                  metrics=['accuracy'])

    model.summary()

    return model

tests.test_model_final(model_final)


print('Final Model Loaded')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_69 (Embedding)     (None, 15, 10)            1990      
_________________________________________________________________
bidirectional_58 (Bidirectio (None, 15, 21)            4032      
_________________________________________________________________
time_distributed_87 (TimeDis (None, 15, 344)           7568      
Total params: 13,590.0
Trainable params: 13,590
Non-trainable params: 0.0
_________________________________________________________________


AssertionError: Wrong output shape. Found output shape (None, 15, 344) using parameters output_sequence_length=21 and french_vocab_size=344

In [236]:
final_rnn = model_final(
    padded_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) + 1
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_70 (Embedding)     (None, 21, 10)            2000      
_________________________________________________________________
bidirectional_59 (Bidirectio (None, 21, 21)            4032      
_________________________________________________________________
time_distributed_88 (TimeDis (None, 21, 346)           7612      
Total params: 13,644.0
Trainable params: 13,644
Non-trainable params: 0.0
_________________________________________________________________


In [237]:
final_rnn.fit(padded_x, preproc_french_sentences, batch_size=4096, epochs=10, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 10s - loss: 4.8218 - acc: 0.3301 - val_loss: 3.8856 - val_acc: 0.4097
Epoch 2/10
110288/110288 [==============================] - 5s - loss: 3.4780 - acc: 0.4082 - val_loss: 3.0603 - val_acc: 0.4093
Epoch 3/10
110288/110288 [==============================] - 5s - loss: 2.8888 - acc: 0.4082 - val_loss: 2.7555 - val_acc: 0.4093
Epoch 4/10
110288/110288 [==============================] - 5s - loss: 2.6786 - acc: 0.4123 - val_loss: 2.5914 - val_acc: 0.4222
Epoch 5/10
110288/110288 [==============================] - 5s - loss: 2.5112 - acc: 0.4342 - val_loss: 2.4005 - val_acc: 0.4564
Epoch 6/10
110288/110288 [==============================] - 5s - loss: 2.3381 - acc: 0.4800 - val_loss: 2.2539 - val_acc: 0.5076
Epoch 7/10
110288/110288 [==============================] - 5s - loss: 2.1663 - acc: 0.5336 - val_loss: 2.0731 - val_acc: 0.5555
Epoch 8/10
110288/110288 [===================

In [238]:
preview_sentence(3,padded_x,final_rnn)

ENGLISH:  the united states is sometimes mild during june , and it is cold in september .
FRENCH:  les ã©tats votre est est est parfois en et et est est est en en en <PAD> <PAD> <PAD> <PAD> <PAD>


## Prediction (IMPLEMENTATION)

In [ ]:
import numpy as np
import os.path
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.sequence import pad_sequences

def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    output_length = max(x.shape[1],y.shape[1])
    x = pad(x,output_length)
    y = pad(y,output_length)
    model = model_final(
        padded_x.shape,
        output_length,
        len(english_tokenizer.word_index) + 1,
        len(french_tokenizer.word_index) + 1
    )
    
    weight_path = 'weights.model_final5.hdf5'
    checkpointer = ModelCheckpoint(
        filepath=weight_path,
        verbose=1, save_best_only=True
    )
    earlystopper = EarlyStopping(min_delta=0.001,patience=100,verbose=1)
    
    if os.path.isfile(weight_path):
        model.load_weights(weight_path)

    model.fit(
        x,y,
        batch_size=4096,
        epochs=1000,
        validation_split=0.2,
        callbacks=[checkpointer, earlystopper]
    )

    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_79 (Embedding)     (None, 21, 10)            2000      
_________________________________________________________________
bidirectional_68 (Bidirectio (None, 21, 21)            4032      
_________________________________________________________________
time_distributed_97 (TimeDis (None, 21, 346)           7612      
Total params: 13,644.0
Trainable params: 13,644
Non-trainable params: 0.0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/1000
110288/110288 [==============================] - 12s - loss: 1.3583 - acc: 0.6754 - val_loss: 1.3358 - val_acc: 0.6807
Epoch 2/1000
110288/110288 [==============================] - 5s - loss: 1.3254 - acc: 0.6794 - val_loss: 1.3299 - val_acc: 0.6790
Epoch 3/1000
110288/110288 [==============================] - 5s - loss: 1.3227 - ac

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.